<a href="https://colab.research.google.com/github/e19166/e19-4yp-Dynamic-Multi-Dimensional-Resource-Orchestration-in-Kubernetes/blob/main/results/service2_cpu_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('service2_cpu_limit_reduction.csv')
df.head()  # to preview the data

,Timestamp,Service,CPU Request,Memory Request,CPU Limit,Memory Limit,Latency,CPU Usage,Memory Usage
0,2025-05-11T17:25:23.846000,service-2-deployment,0.1,134217728,0.2,268435456,0.000148,0.183241,10010624
1,2025-05-11T17:25:53.917000,service-2-deployment,0.1,134217728,0.2,268435456,0.000148,0.187181,10010624
2,2025-05-11T17:26:23.989000,service-2-deployment,0.1,134217728,0.2,268435456,0.000148,0.187187,10010624
3,2025-05-11T17:26:54.066000,service-2-deployment,0.1,134217728,0.2,268435456,0.000148,0.187076,10010624
4,2025-05-11T17:27:24.135000,service-2-deployment,0.1,134217728,0.2,268435456,0.000148,0.187079,10010624
